In [9]:
import numpy as np 
import scipy #intégrer et générer les cdf et pdf de beta 

## Binomial bandit 

In [2]:
#générer les theta 
k = 3
theta = np.random.uniform(0,1,k)

In [3]:
def f(a,theta): 
    return np.random.binomial(1, theta[a])

In [4]:
# case mu_a(theta)=theta_a (binomial bandit)
def I(a,theta): 
    if theta[a]== max(theta): 
        return 1
    return 0

In [12]:
# Python code for computing Equation (11) by quadrature.
def compute_probopt(y, n): 
    k = len(y)
    ans = np.zeros(k)
    for i in k: 
        def fct_to_integrate(x):
            r = scipy.stats.beta.cdf(x, y[i]+1, n[i]-y[i+1])
            for j in k: 
                if i!=j: 
                    r = r * scipy.stats.beta.cdf(x, y[j]+1, n[j]-y[j+1])
            return r
        ans[i] = scipy.integrate.quad(fct_to_integrate, 0, 1)
    return ans                

In [13]:
# Function to simulate posterior samples
def sim_post(y, n, ndraws):
    k = len(y)
    ans = np.zeros((ndraws, k))
    no = n - np.array(y)
    for i in range(k):
        ans[:, i] = beta.rvs(y[i] + 1, no[i] + 1, size=ndraws)
    return ans

# Function to calculate the probability of a winner
def prob_winner(post):
    k = post.shape[1]
    w = np.bincount(np.argmax(post, axis=1), minlength=k)
    return w / np.sum(w)

# Function to compute the winning probabilities
def compute_win_prob(y, n, ndraws):
    post = sim_post(y, n, ndraws)
    return prob_winner(post)